In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import time
import random
import os
from math import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as MAE
from joblib import dump, load

In [ ]:
data_path = '../data/'
df2 = pd.read_csv(data_path+'pricing_requests_34reqforlistdates.csv')
df1 = pd.read_csv(data_path+'pricing_requests_20req_eachdate.csv')
df3 = pd.read_csv(data_path+'pricing_requests_92req_sofia.csv')
df4 = pd.read_csv(data_path+'pricing_requests_92req_valletta.csv')
df5 = pd.read_csv(data_path+'pricing_requests_100req.csv')
df6 = pd.read_csv(data_path+'pricing_requests_500randomreq.csv')
#df7 = pd.read_csv('pricing_requests_500randomreq_date1.csv')
df8 = pd.read_csv(data_path+'pricing_requests_cities_3000_3100req.csv')
#df9 = pd.read_csv('455_pricing_requests_date22_unzipped.csv')
#df10 = pd.read_csv('pricing_requests_500_random.csv')

In [ ]:
df_complete_january_1 = pd.concat([df1, df2, df3, df4, df5, df6,  df8])#pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10,
                                 #df11, df12, df13, df14, df15, df16, df17, df18, df19])

In [ ]:
df_complete_january = df_complete_january_1.drop_duplicates()

In [ ]:
features_hotels = pd.read_csv(data_path+'features_hotels.csv')
X_test = pd.read_csv(data_path+'test_set.csv')

In [ ]:
#it's the same as in the original column
del features_hotels['city']

In [ ]:
df_complete_features = df_complete_january.join(features_hotels.set_index('hotel_id'), on='hotel_id')

In [ ]:
X_test_features = X_test.join(features_hotels.set_index('hotel_id'), on='hotel_id')

In [ ]:
df_complete_features

In [ ]:
X_test_features

In [ ]:
X_test_features = X_test_features.drop(X_test_features.columns[[0,1]], axis=1)
del X_test_features['avatar_id']
del X_test_features['hotel_id']

In [ ]:
X_test_features

In [ ]:
df_complete_features = df_complete_features.drop(df_complete_features.columns[[0]], axis=1)
del df_complete_features['avatar_id']
del df_complete_features['hotel_id']

In [ ]:
df_complete_features

In [ ]:
cities = ['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'sofia', 'valletta', 'vienna', 'vilnius']

#calculate the mean prices for each city
mean_cities = []
for city in cities:
  #isolate the city
  rslt_df = df_complete_features[df_complete_features['city'] == city]
  #rslt_df['city'] =np.mean(rslt_df['price'])
  mean_cities.append(np.mean(rslt_df['price']))
total = np.array(mean_cities).sum()
mean_cities = np.array(mean_cities)/total
print(mean_cities)
city_mean = dict(zip(cities,mean_cities))

In [ ]:
languages = ['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch',
             'estonian', 'finnish', 'french', 'german', 'greek', 'hungarian', 'irish', 'italian',
             'latvian', 'lithuanian', 'luxembourgish', 'maltese', 'polish', 'portuguese', 'romanian',
             'slovakian', 'slovene', 'spanish', 'swedish']

#calculate the mean prices for each language
mean_languages = []
for language in languages:
    rslt_df = df_complete_features[df_complete_features['language'] == language]
    mean_languages.append(np.mean(rslt_df['price']))
total = np.array(mean_languages).sum()
mean_languages = np.array(mean_languages)/total
print(mean_languages)
language_mean = dict(zip(languages,mean_languages))

In [ ]:
groups = ['Morriott International','Chillton Worldwide','Independant','Accar Hotels','Yin Yang','Boss Western']

#calculate the mean prices for each city
mean_groups = []
for group in groups:
    rslt_df = df_complete_features[df_complete_features['group'] == group]
    mean_groups.append(np.mean(rslt_df['price']))
total = np.array(mean_groups).sum()
mean_groups = np.array(mean_groups)/total
print(mean_groups)
group_mean = dict(zip(groups,mean_groups))

In [ ]:
brands = ['Chill Garden Inn','Morriot','Royal Lotus','Corlton','Independant','Ibas','Navatel','Ardisson',
'Marcure','Safitel','Boss Western','J.Halliday Inn','8 Premium','Tripletree','CourtYord','Quadrupletree']

#calculate the mean prices for each city
mean_brands = []
for brand in brands:
    rslt_df = df_complete_features[df_complete_features['brand'] == brand]
    mean_brands.append(np.mean(rslt_df['price']))
total = np.array(mean_brands).sum()
mean_brands = np.array(mean_brands)/total
print(mean_brands)
brand_mean = dict(zip(brands,mean_brands))

In [ ]:
print(city_mean)
pd.DataFrame([city_mean]).to_csv("../processed_data/city_mean.csv")
print(group_mean)
pd.DataFrame([group_mean]).to_csv("../processed_data/group_mean.csv")
print(brand_mean)
pd.DataFrame([brand_mean]).to_csv("../processed_data/brand_mean.csv")
print(language_mean)
pd.DataFrame([language_mean]).to_csv("../processed_data/language_mean.csv")


In [ ]:
def target_embedding(df_original):
    new_df = df_original.copy()
    
    #city
    dfs = []
    for city in cities:
        rslt_df = new_df[new_df['city'] == city]
        rslt_df['city'] = city_mean[city]
        dfs.append(rslt_df)
    new_df = pd.concat(dfs)

    
    #language
    dfs = []
    for language in languages:
        rslt_df = new_df[new_df['language'] == language]
        rslt_df['language'] = language_mean[language]
        dfs.append(rslt_df)
    new_df = pd.concat(dfs)

    
    #group
    dfs = []
    for group in groups:
        rslt_df = new_df[new_df['group'] == group]
        rslt_df['group'] = group_mean[group]
        dfs.append(rslt_df)
    new_df = pd.concat(dfs)
    
    
    #brand
    dfs = []
    for brand in brands:
        rslt_df = new_df[new_df['brand'] == brand]
        rslt_df['brand'] = brand_mean[brand]
        dfs.append(rslt_df)
    new_df = pd.concat(dfs)
    new_df = new_df.sort_index(ascending=True)
    
    return new_df

In [ ]:
df_complete_fearures = target_embedding(df_complete_features)
X_test_features = target_embedding(X_test_features)

In [ ]:
df_complete_fearures

In [ ]:
df_complete_fearures.to_csv('../processed_data/entire_df_complete_embed_feature.csv')

In [ ]:
X_test_features

In [ ]:
X_test_features.to_csv('../processed_data/entire_df_test_embed2.csv')

In [ ]:
X = df_complete_fearures.copy()
del X['price']
X.to_csv('../processed_data/Train_feature.csv',index=False)
y = df_complete_fearures['price']
y.to_csv('../processed_data/label_feature.csv',index=False)

In [ ]:
df_complete_fearures 

In [ ]:
'''
df_complete_fearures_dup = df_complete_fearures.copy()
del df_complete_fearures_dup['price']
df_complete_fearures_dup.drop_duplicates()
print(len(df_complete_fearures_dup))
'''

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
'''from xgboost import XGBRegressor

XGBregressor = XGBRegressor(learning_rate=0.7,
                          subsample = 1,
                          max_depth=1000
                         )
XGBregressor.fit(X_train,y_train)

Y_predict_val = XGBregressor.predict(X_val)
MSE(y_val, Y_predict_val)'''

In [ ]:
from xgboost import XGBRegressor
regressor2 = XGBRegressor(learning_rate=0.01,  
                      colsample_bytree = 1,
                      subsample = 1,
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=10, 
                      gamma=0)
regressor2.fit(X_train,y_train)

In [ ]:
# evaluate using train test
Y_predict_val_train = regressor2.predict(X_train)
MSE(y_train, Y_predict_val_train)

In [ ]:
Y_predict_val = regressor2.predict(X_val)
MSE(y_val, Y_predict_val)

In [ ]:
X_test_features = X_test_features[['stock', 'city', 'date', 'language', 'mobile', 'group', 'brand', 'parking', 'pool', 'children_policy']]

In [ ]:
X_test_features.to_csv('../processed_data/test_features.csv')
Y_pred_34 = regressor2.predict(X_test_features)

In [ ]:
Y_pred_34 = pd.DataFrame(Y_pred_34)
Y_pred_34.to_csv("../predictions/Y_pred_34.csv")

In [ ]:
# Export models using joblib 

dump(regressor2, '../models/regressor_xgb.joblib') 

In [ ]:
import session_info
session_info.show(write_req_file=True)

In [ ]:
#Building stacking model
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor,RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
estimators = [('xg', XGBRegressor()),('rf',RandomForestRegressor(n_estimators=150,random_state=42)),('gbs',GradientBoostingRegressor())]
reg = StackingRegressor(estimators=estimators,final_estimator=AdaBoostRegressor())
reg.fit(X_train,y_train)

In [ ]:
Y_predict_val_train_stack = reg.predict(X_train)
MSE(y_train, Y_predict_val_train_stack)

In [ ]:
X_test_features.to_csv('../processed_data/test_features.csv')
Y_pred_34_stack = reg.predict(X_test_features)
Y_pred_34_stack = pd.DataFrame(Y_pred_34_stack)
Y_pred_34_stack.to_csv("../predictions/Y_pred_34_stack.csv")

In [ ]:
dump(reg, '../models/regressor_stacking.joblib') 